# FILTRAGE DES JEUX DE DONNEES

In [1]:
from utils.io import dataframe_viewer, dict_viewer, data_merger, data_validation, na_line_drop, find_bh_synonyms, \
choose_bh_synonym, gen_id_from_ech, value_replication, find_borehole_by_position

import re, os
import json
import numpy as np
import pandas as pd
import geopandas as gpd
from definitions import ROOT_DIR

In [2]:
save_dir = ROOT_DIR + '/CF_data/Result_traitem/fusion_finale/filtered/'

In [3]:
# os.system(f"rm -f {ROOT_DIR}/utils/boreholes_synonyms/bh_syn_distmax_0-6.py")
os.system(f"rm -rf {save_dir}") 
os.system(f"mkdir {save_dir}") 

0

In [4]:
def list_diff(li1, li2):
    return list(set(li1) - set(li2)) + list(set(li2) - set(li1))

# Lecture de données de référence sur les forages

* Données issues des fiches des logs de forage

In [5]:
import importlib.machinery
import importlib.util

loader = importlib.machinery.SourceFileLoader('known_bhs', '/home/yanathan/Projects/GSDMA/CF_data/known_boreholes.py' )
spec = importlib.util.spec_from_loader('known_bhs', loader)
known_bhs = importlib.util.module_from_spec(spec)
loader.exec_module(known_bhs)
log_bhs = known_bhs.known_boreholes

In [6]:
df_log_bh = pd.DataFrame()

In [7]:
idx = 0
for k, v in log_bhs.items():
    df_log_bh.loc[idx, 'ID'] = k
    for p, q in v.items():
        df_log_bh.loc[idx, p] = q
    idx += 1

In [8]:
dataframe_viewer(df_log_bh, rows=5, un_val='ID')

Rows : 65, columns : 8, Unique values on cols: {'ID': 65}


interactive(children=(IntSlider(value=5, description='rows', max=65, min=5, readout=False), IntSlider(value=8,…

* Données issues des shapefiles

In [9]:
shapef_dir = ROOT_DIR+"/CF_data/Data_UMONS/SIG/"
shp1 = gpd.read_file(shapef_dir+"Pilote_v8.0.shp")
shp2 = gpd.read_file(shapef_dir+"sondages_v2017-05-18.shp")

In [10]:
shp1['X'] = shp1.geometry.x
shp1['Y'] = shp1.geometry.y

In [11]:
shp2.rename(columns={'Ref_sond':'ID','XL72':'X','YL72':'Y','Date':'Date_for','Prof_sond':'Long_for'}, inplace=True)
shp1.rename(columns={'Ref':'ID'}, inplace=True)

In [12]:
shp1 = shp1[['ID', 'X', 'Y']]
shp2 = shp2[['ID', 'X', 'Y', 'Z', 'Refus', 'Date_for', 'Long_for']]

In [13]:
shp_df, conflict_df=data_merger(shp1, shp2, how='outer', on=['ID'], dist_max=.5)

### Traitements et fusion des données de base

In [14]:
idx = shp_df.query('ID.str.contains("^P\d")').index
shp_df.loc[idx, 'ID'] = shp_df.loc[idx, 'ID'].apply(lambda x: x.replace('P','F'))

Fusion

In [15]:
bh_base, conflict_df=data_merger(df_log_bh, shp_df, how='outer', on=['ID'], dist_max=0.5)

Conflict values present. Please resolve this manually !


In [16]:
dataset = bh_base
data_validation(overall_data=dataset, conflict_data=conflict_df, index_col='index', 
                valid_dict={'Date_for_y':list(conflict_df.index)})

if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

all conflicts have been fixed!


In [17]:
dataframe_viewer(rows=5, un_val=['ID'], df=bh_base)#.query('ID.str.contains("^P\d")'))

Rows : 186, columns : 9, Unique values on cols: {'ID': 186}


interactive(children=(IntSlider(value=5, description='rows', max=186, min=5, readout=False), IntSlider(value=9…

# Lecture des jeux de données issues du prétraitement final

In [18]:
bh = pd.read_csv(ROOT_DIR+'/CF_data/Result_traitem/fusion_finale/Boreholes.csv', delimiter=',')
litho = pd.read_csv(ROOT_DIR+'/CF_data/Result_traitem/fusion_finale/Lithologies.csv', delimiter=',')
samp = pd.read_csv(ROOT_DIR+'/CF_data/Result_traitem/fusion_finale/Samples.csv', delimiter=',')
meas = pd.read_csv(ROOT_DIR+'/CF_data/Result_traitem/fusion_finale/Measures.csv', delimiter=',')
eqp = pd.read_csv(ROOT_DIR+'/CF_data/Result_traitem/fusion_finale/Equipments.csv', delimiter=',')
ukn = pd.read_csv(ROOT_DIR+'/CF_data/Result_traitem/fusion_finale/Unknown.csv', delimiter=',')

In [19]:
data_dict = {'boreholes':bh, 'lithologies':litho, 'samples':samp, 'measures':meas, 'equipments':eqp, 'unknows':ukn}
data_coi = {'boreholes':[], 'lithologies':[], 'samples':[], 'measures':[], 'equipments':[], 'unknows':[]}

In [20]:
for d_key in data_dict.keys():
    coi = []
    for c in ['ID','ID_ech','X','Y','Z','Long_for','Diam_for','Date_','_top','_base','_epai','Description']:
        for col in data_dict[d_key].columns:
            if '_' in c and re.search(c, col, re.I):
                coi.append(col)
            elif c == col:
                coi.append(col)

    data_coi.update({d_key: coi})

In [21]:
dict_viewer(data_coi)

boreholes,"['ID', 'X', 'Y', 'Z', 'Long_for', 'Diam_for', 'Date_for']"
lithologies,"['ID', 'X', 'Y', 'Z', 'Long_for', 'Date_for', 'Litho_top', 'Litho_base', 'Litho_epais', 'Description']"
samples,"['ID', 'ID_ech', 'X', 'Y', 'Z', 'Long_for', 'Date_ech', 'Date_for', 'Pol_top', 'Ech_top', 'Pol_base', 'Ech_base', 'Description']"
measures,"['ID', 'ID_ech', 'X', 'Y', 'Z', 'Date_mes', 'Date_for']"
equipments,"['ID', 'X', 'Y', 'Z', 'Diam_for', 'Date_for', 'Equip_top', 'Equip_base', 'Equip_epais']"
unknows,"['ID', 'X', 'Y', 'Z', 'Long_for', 'Date_for']"


### Traitements et fusion des jeux de données

* Forages

In [22]:
key = 'boreholes'
data_dict[key].insert(0, 'ID', data_dict[key].pop('ID'))

In [23]:
idx = data_dict[key].query('ID.str.contains("^P\d")').index
data_dict[key].loc[idx, 'ID'] = data_dict[key].loc[idx, 'ID'].apply(lambda x: x.replace('P','F'))

Fusion avec les données de base

In [24]:
bh_all, conflict_df=data_merger(bh_base, data_dict[key], how='outer', on=['ID'], dist_max=1)

Conflict values present. Please resolve this manually !


In [25]:
dataset = bh_all
data_validation(overall_data=dataset, conflict_data=conflict_df, index_col='index', 
                valid_dict={'Refus_y':list(conflict_df.index), 'Date_for_x':list(conflict_df.index), 
                            'Long_for_x':list(conflict_df.index), 'Diam_for_x':list(conflict_df.index)})

if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

all conflicts have been fixed!


In [26]:
sufx = ['sup', 'prof', 'inf', '/\dM(\*)?']
prefx = ['eau forage ']
id_reg = '\s*(?P<id>(?:^canne |Piezair | bloqué$)*\w*\d+\w*)\s*'
keys = ['boreholes','lithologies','samples','measures','equipments','unknows']

In [27]:
bh_all = gen_id_from_ech(bh_all, id_ech_col='ID', suffixes=sufx, prefixes=prefx, capture_regex=id_reg, 
                         regex_skip='bloqué')
bh_all.drop(columns='ID_ech', inplace=True)

In [28]:
bh_all.loc[bh_all.query('Diam_for==75').index, 'Diam_for'] = 0.075

In [29]:
chg_bh_id = {'224':[213],'_224_':[214],'226':[215],'306':[216],"F27dM":[192,193],'F27aM':[189],'F26M':[187,188,718],
         'F26':[719],'F25M':[185,186,613],'F25':[612],'F24aM':[182],'F24bM':[183,184],'F23M':[180,181,716],'F17dM':[228],
         'F23':[717],'F22M':[178,179],'50':[206],'51':[207],'51':[208,810],'F2M':[329],'F11M':[334],'F12M':[335],
         'F13M':[336],'F14M':[225],'F15bM':[226],'F18bM':[169],"F18c":"F18cM",'F16M':[227],'F19M':[172,173,714,715],}

for k, v in chg_bh_id.items(): 
    if isinstance(v, list):
        bh_all.loc[v, 'ID'] = k
    elif isinstance(v, str):
        bh_all.loc[bh_all.query(f'ID=="{k}"').index, 'ID'] = v

In [30]:
data_dict[key] = bh_all

In [31]:
bh_cols = ['ID','X','Y','Z','Long_for','Diam_for','Date_for']#,'Refus','Societe','Zone','Etude','Method']
bhs = bh_all[bh_cols]

* Lithologies

In [32]:
key = 'lithologies'
data_dict[key].insert(0, 'ID', data_dict[key].pop('ID'))

In [33]:
litho_cols = ['ID','X','Y','Z','Date_for','Long_for','Litho_top','Litho_base','Description']

In [34]:
# skip trenches data
data_dict[key] = data_dict[key].drop(index=data_dict[key].query('ID.str.contains("^T")').index)[litho_cols]

In [35]:
dataframe_viewer(rows=5, un_val=['ID'], df=data_dict[key])

Rows : 1845, columns : 9, Unique values on cols: {'ID': 401}


interactive(children=(IntSlider(value=5, description='rows', max=1845, min=5, readout=False), IntSlider(value=…

correction d'incohérences sur les ID

In [36]:
chg_bh_id = {"F219a":'F219B', "F22":'F22A', "F48a":'F48B', "F195b":'F195B', "F205a":'F205A', "F50a":'F50B', 
    "F113a":'F113A', "F114a":'F114A', "F114b":'F114B', "F115a":'F115A', "F115b":'F115B', "F116a":'F116A', 
    "F116b":'F116B', "F117a":'F117A', "F117b":'F117B', "F118a":'F118A', "F118b":'F118B', "F119a":'F119A', 
    "F120":'F120A', "F121":'F121A', "F122":'F122A', "F128":'F128B', "F130":'F130B', "F134":'F134B', 
    "F137":'F137B', "F139":'F139C', "F140":'F140C', "F141":'F141B', "F143":'F143B', "F115b":'F115B', 
    "F115a":'F115A', "F157":'F157B', "F159":'F159B', "F160":'FP160', "F164a":'F164A', "F164b":'F164B', 
    "F179":'F179C', "F183":'F183B', "F184":'F184C', "F188":'F188B', "F194":'F194B', "F195a":'F195A', 
    "F195b":'F195B', "F200a":'F200A', "F205a":'F205A', "F27a":'F27A', "F28a":'F28A', "F37a":'F37B', 
    "F514a":'F514b', "F51a":'F51B', "F219a":'F219B', "F9a":'F9A', "F9b":'F9B', "F15a":'F15A', "F14":'FP14',
    "F14":'FP14', "F521a":'F521A', "F525a":'F525', "F531a":'F531', "F542a":'F542A', "F595":'FP595',
    "F63":'FP63', "F76c":'FP76', "F76a":'F76B', "F76b":'F76C', "F78a":'F78B', "F88a":'F88B', 
    "F91bis":'PZ91Bis', "F94a":'F94B', "F94b":'F94C', "F97a":'F97B', "F97b":'F97C'}

for k, v in chg_bh_id.items(): 
    data_dict[key].loc[data_dict[key].query(f'ID=="{k}"').index, 'ID'] = v

Fusion avec les données de forages

In [37]:
dat, conflict_df=data_merger(bhs, data_dict[key], how='outer', on=['ID'], dist_max=0.5)

Conflict values present. Please resolve this manually !


In [38]:
dataset = dat
data_validation(overall_data=dataset, conflict_data=conflict_df, index_col='index', 
                valid_dict={'Date_for_x':list(conflict_df.index), 'Long_for_y':list(conflict_df.index)})

if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

all conflicts have been fixed!


In [39]:
crit_cols = ['ID','X','Y']
#pass_cols = crit_cols + ['Litho_top','Litho_base','Litho_epais','Description']
copy_cols = ['Z','Date_for','Long_for','Diam_for','Societe']
data = value_replication(dat, crit_cols=crit_cols, pass_cols=None, copy_cols=copy_cols)

In [40]:
data = data.query('X==X and Litho_base==Litho_base')

In [41]:
data = data.drop_duplicates(['ID','X','Y','Z','Litho_top','Litho_base','Description'])[litho_cols+['Diam_for']]
data.sort_values(['ID','Litho_top'], inplace=True)
data.reset_index(drop=True, inplace=True)

In [42]:
dataframe_viewer(rows=5, un_val=['ID'], df=data)#.query('Litho_base!=Litho_base'))

Rows : 1890, columns : 10, Unique values on cols: {'ID': 397}


interactive(children=(IntSlider(value=5, description='rows', max=1890, min=5, readout=False), IntSlider(value=…

In [43]:
litho_all = data
data_dict[key] = data

* Analyses

In [44]:
key = 'samples'
data_dict[key].insert(0, 'ID', data_dict[key].pop('ID'))
data_dict[key] = data_dict[key].query('ID==ID')

In [45]:
samp_cols = ['ID','ID_ech','Type_ech','X','Y','Z','Date_for','Long_for','Ech_top','Ech_base','Description']
samp_cols = samp_cols + [x for x in data_dict[key].columns if x not in samp_cols]
samp_cols.remove('level_0')

In [46]:
# skip trenches data
data_dict[key] = data_dict[key].drop(index=data_dict[key].query('ID.str.contains("^T")').index)[samp_cols]

In [47]:
data = data_dict[key].query('Ech_base==Ech_base') # water samples are wipe !
data.sort_values(['ID','Ech_top'], inplace=True)
data.reset_index(drop=True, inplace=True)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [48]:
dataframe_viewer(rows=5, un_val=['ID'], df=data)#.query('Type_ech=="Sol"')[samp_cols])

Rows : 1496, columns : 219, Unique values on cols: {'ID': 307}


interactive(children=(IntSlider(value=5, description='rows', max=1496, min=5, readout=False), IntSlider(value=…

correction d'incohérences sur les ID

In [49]:
chg_bh_id = {"224b":'224','F9M':"F9bM",'F9B':[1258,1260,1308],'F9bM':[1259,1316],'F6M':[1080,1077],'F17dM':[512],
             'F5M':[915,919,920],'F4M':[806,810,913],'F4':[912,914],'F3M':[732,734],'F26M':[699,701,703,705],
             'F25M':[687,692,694],'F24bM':[686],'F23M':[675,677],"F22":'F22M',"F27d":"F27dM",'F13M':[285,290,292],
             'F2M':[575,579,581],'F1aM':[98,100,570,571],'F10M':[106],'F11M':[169],'F12M':[233,235],'F14M':[333],
             'F15bM':[405,406,407],'F18bM':[547],"F18c":"F18cM",'F16M':[411,412],'F19M':[554,556]}

for k, v in chg_bh_id.items(): 
    if isinstance(v, list):
        data.loc[v, 'ID'] = k
    elif isinstance(v, str):
        data.loc[data.query(f'ID=="{k}"').index, 'ID'] = v

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Fusion avec les données de forages

In [50]:
dat, conflict_df=data_merger(bhs, data, how='outer', on=['ID'], dist_max=0.5)

Conflict values present. Please resolve this manually !


In [51]:
dataset = dat
data_validation(overall_data=dataset, conflict_data=conflict_df, index_col='index', 
                valid_dict={'Date_for_x':list(conflict_df.index), 'Long_for_x':[118,120]})

if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

all conflicts have been fixed!


In [52]:
data = dat.query('ID_ech==ID_ech').sort_values(['ID','Ech_top'])
data.drop(['Pol_top','Pol_base','Refus'], axis=1, inplace=True)

In [53]:
crit_cols = ['ID','X','Y']
#pass_cols = crit_cols + ['Litho_top','Litho_base','Litho_epais','Description']
copy_cols = ['Z','Date_for','Date_ech','Long_for','Diam_for','Societe']
data = value_replication(data, crit_cols=crit_cols, pass_cols=None, copy_cols=copy_cols)

In [54]:
data = data.query('X==X and Ech_base==Ech_base')

In [55]:
samp_cols = ['ID','ID_ech','Type_ech','X','Y','Z','Date_for','Diam_for','Long_for','Ech_top','Ech_base','Description']
samp_cols = samp_cols + [x for x in data.columns if x not in samp_cols]

In [56]:
data = data.drop_duplicates(['ID','X','Y','Z','Ech_top','Ech_base','Description'])[samp_cols]
data.sort_values(['ID','Ech_top'], inplace=True)
data.reset_index(drop=True, inplace=True)

In [57]:
dataframe_viewer(rows=5, un_val=['ID'], df=data)#.query('X!=X and Ech_base!=Ech_base'))

Rows : 1515, columns : 217, Unique values on cols: {'ID': 295}


interactive(children=(IntSlider(value=5, description='rows', max=1515, min=5, readout=False), IntSlider(value=…

In [58]:
samp_all = data
data_dict[key] = data

### suppression des doublons du jeu de données des forages

Fusion et récupération de coordonnées

In [59]:
dat, conflict_df=data_merger(bh_all, litho_all[bh_cols], how='outer', on=['ID'], dist_max=0.5)

Conflict values present. Please resolve this manually !


In [60]:
dataset = dat
data_validation(overall_data=dataset, conflict_data=conflict_df, index_col='index', 
                valid_dict={'Date_for_y':list(conflict_df.index), 'Long_for_y':list(conflict_df.index),
                           'Z_x':list(conflict_df.index)})

if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

all conflicts have been fixed!


In [61]:
data, conflict_df=data_merger(dat, samp_all[bh_cols], how='outer', on=['ID'], dist_max=0.5)

Conflict values present. Please resolve this manually !


In [62]:
dataset = data
data_validation(overall_data=dataset, conflict_data=conflict_df, index_col='index', 
                valid_dict={'Date_for_y':list(conflict_df.index), 'Long_for_y':list(conflict_df.index),
                           'Z_y':list(conflict_df.index)})

if 'level_0' in dataset.columns:
    if 'index' in dataset.columns:
        dataset.drop(columns='index', inplace=True)
    dataset.rename(columns={'level_0':'index'}, inplace=True)

all conflicts have been fixed!


Suppression des doublons

In [63]:
data = data.query('X==X')

In [64]:
crit_cols = ['ID','X','Y']
#pass_cols = crit_cols + ['Litho_top','Litho_base','Litho_epais','Description']
copy_cols = list(data.columns)
data = value_replication(data, crit_cols=crit_cols, pass_cols=None, copy_cols=copy_cols)

In [65]:
dat = data.drop_duplicates(['ID','X','Y'])
dat.sort_values('ID', inplace=True)
dat.reset_index(drop=True, inplace=True)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [66]:
data_dict['boreholes'] = dat.query('X==X').copy()

####  $\color{red}{\textbf{Sauvegarde du jeu de données}}$

In [67]:
for k in list(data_dict.keys())[:3]:
    data_dict[k].to_csv(save_dir + f'{k.capitalize()}.csv', index=False)